In [3]:
import numpy as np
import tensorflow as tf
from data_gen import *
import random
import matplotlib.pyplot as plt
import numpy as np
import math

In [4]:
num_features = 1
batch_size = 3
state_size = 2
datatype = tf.float64
num_of_operations = 2


gloabal_seed = round(random.random()*100000)
np.random.seed(gloabal_seed)
tf.set_random_seed(gloabal_seed)
sess = tf.InteractiveSession()

x,y = samples_generator(np_add, (batch_size, num_features) , (-1,1), gloabal_seed)

batchX = tf.Variable(x, dtype=datatype)
state  = tf.Variable(np.random.rand(batch_size, state_size), dtype=datatype)
W      = tf.Variable(np.random.rand(state_size+num_features, state_size), dtype=datatype)
W2     = tf.Variable(np.random.rand(state_size, num_of_operations),dtype=datatype)
W3     = tf.Variable(np.random.rand(num_of_operations, num_features),dtype=datatype)

batchX.initializer.run()
state.initializer.run()
W.initializer.run()
W2.initializer.run()
W3.initializer.run()

In [ ]:
list(zip(x, y))

In [ ]:

def custom_softmax(x, base = 1):
    maxx = tf.reduce_max(x, axis=1, keep_dims=True)
    print("maxx")
    print(maxx.eval())
    maxg = (x - maxx)
    print("maxg")
    print(maxg.eval())
    #powx = tf.pow(maxg, tf.cast(-base, datatype))
    maxg =tf.nn.relu(maxg)
    powx = maxg
    for i in range(base):
        powx = tf.sqrt(powx)
    #reduced  = tf.reduce_sum(powx, axis=1, keep_dims=True)

    return powx
#tf.maximum

'''
def custom_softmax( x,base = 380):
        maxx = tf.reduce_max(x, axis=1, keep_dims=True)
        powx = tf.pow(tf.cast(base, datatype), x-maxx)
        reduced  = tf.reduce_sum(powx, axis=1, keep_dims=True)
        maxg = powx/ reduced
        return maxg
'''

def calc_gradW3(W3, softmax):
    shape = W3.get_shape().as_list()
    der = tf.Variable(np.zeros(shape), dtype=datatype)
    der.initializer.run()
    for i in range(shape[0]):
        for j in range(shape[1]):
            d_temp = tf.Variable(np.zeros(shape), dtype=datatype)
            d_temp.initializer.run()
            tf.assign( d_temp[i,j], 1).eval()
            der_temp = tf.reduce_sum(- tf.matmul(softmax, d_temp))
            tf.assign( der[i,j], der_temp).eval()
    return der

In [ ]:
def run_pass(batchX, state):
    X_and_state_con = tf.concat([batchX, state], 1)
    print("X_and_state_con")
    print(X_and_state_con.eval())
    layer1          = tf.matmul(X_and_state_con, W)
    print("layer1")
    print(layer1.eval())
    #next_state      = tf.nn.sigmoid(layer1)
    next_state      = tf.nn.relu(layer1)
    print("next_state")
    print(next_state.eval())
    state_dropped = tf.layers.dropout(next_state, 0, training = True)
    #print("state_dropped")
    #print(state_dropped.eval())
    logits = tf.matmul(state_dropped, W2)
    print("logits")
    print(logits.eval())
    #softmax = custom_softmax(logits, 1)
    softmax = tf.nn.softmax(500*logits)
    #print("cust_softmax")
    #print(cust_softmax.eval())
    print("softmax")
    print(softmax.eval())
    #softmax_loss = skewed_sigmoid_der_tf(softmax, num_of_operations)
    #print("softmax_l2_losts")
    #print(softmax_loss.eval())
    next_x = tf.matmul(softmax, W3)
    print("next_x")
    print(next_x.eval())
    loss_non_red = y - next_x
    loss = tf.reduce_sum(loss_non_red)
    print("loss")
    print(loss.eval())
    grads  = tf.gradients(loss, [W,W2,W3])
    print("weights")
    [print(w.eval()) for w in [W,W2,W3]]
    print("grads_tf")
    [print(500*grad.eval()) for grad in grads]
    #loss fn
    #softmax = e^x/sum(e^x)
    #loss = reduced (y - softmax(relu([x;state_prev] * W1) * W2) * W3)
    print("dev_loss_w3")
    h = 0.001
    dev_loss_w3 = calc_gradW3(W3, softmax)
    print(dev_loss_w3.eval())
    return next_x, next_state

In [ ]:
state1 = run_pass(batchX, state)[1].eval()

In [ ]:
state2 = run_pass(batchX, state1)[1].eval()

In [ ]:
state3 = run_pass(batchX, state2)[1].eval()

In [ ]:
#use derivative of sigmoid to penelise for values other than 1 or 0
def sigmoid_der(x, scale = 10):
    shifted_x = x - 0.5
    scaled_x = scale*shifted_x
    res = tf.sigmoid(scaled_x)*(1 - tf.sigmoid(scaled_x))
    return (res - 0.01)*scale**3

In [ ]:
def skewed_dist(x, scale=10):
    mean = 0.5
    str_d = mean/3.5
    const = 1/tf.sqrt(2*math.pi*(str_d**2))
    power = ((x-mean)**2) / (2*str_d**2)
    return const*math.exp(-power)*scale**2

In [ ]:
def skewed_sigmoid_der(x, scale = 10, num_ops = 3):
    shifted_x = x - 0.5
    scaled_x = scale*shifted_x
    nom = tf.exp(-scaled_x)+num_ops*tf.exp(-num_ops*scaled_x)+(num_ops+1)*tf.exp(-(num_ops+1)*scaled_x)
    denom = tf.square(tf.exp(-scaled_x) + 1)*tf.square(tf.exp(-num_ops*scaled_x) + 1)
    res = nom/denom
    return (res - 0.01)*scale**3

def sigmoid(x, scale = 10, num_ops = 3):
    shifted_x = x - 0.5
    scaled_x = scale*shifted_x
    return tf.sigmoid(scaled_x)*tf.sigmoid(10*scaled_x)

In [ ]:
def plot(fn, ops):
    t = np.arange(0.0, 1.01, 0.01)
    s = [fn(val, ops).eval() for val in t]
    plt.plot(t, s)

    plt.xlabel('softmax')
    plt.ylabel('penalty')
    plt.title(fn.__name__ + " "+str(ops))
    plt.grid(True)
    plt.show()

In [ ]:
def skewed_sigmoid_der_py(x, num_ops = 3, scale = 10):
    worst_case = 1 / num_ops
    shifted_x = x - worst_case + 0.02
    scaled_x = scale*shifted_x
    nom = scale*math.exp(-scaled_x)+scale*num_ops*math.exp(-num_ops*scaled_x)
    #denom = math.square(math.exp(-scaled_x) + 1)*math.square(math.exp(-(num_ops-1)*scaled_x) + 1)
    denom = ((math.exp(-scaled_x) + 1)**2) * ((math.exp(-(num_ops-1)*scaled_x) + 1)**2)
    res = nom/denom
    return worst_case*((res)*scale**3)

def skewed_sigmoid_der_tf(x, num_ops = 3, scale = 10):
    worst_case = 1 / num_ops
    shifted_x = x - worst_case + 0.02
    scaled_x = scale*shifted_x
    nom = scale*tf.exp(-scaled_x)+scale*num_ops*tf.exp(-num_ops*scaled_x)
    denom = tf.square(tf.exp(-scaled_x) + 1)*tf.square(tf.exp(-(num_ops-1)*scaled_x) + 1)
    res = nom/denom
    return worst_case*((res - 0.01)*scale**3)

In [ ]:
plot(skewed_sigmoid_der_py, 2)
plot(skewed_sigmoid_der_py, 3)
plot(skewed_sigmoid_der_py, 4)
plot(skewed_sigmoid_der_py, 5)
plot(skewed_sigmoid_der_py, 6)

In [ ]:
plot(skewed_sigmoid_der_tf, 2)
plot(skewed_sigmoid_der_tf, 3)
plot(skewed_sigmoid_der_tf, 4)
plot(skewed_sigmoid_der_tf, 5)
plot(skewed_sigmoid_der_tf, 6)

In [ ]:
t = np.arange(0.0, 1.01, 0.01)
for val in t:
    print (val, skewed_sigmoid_der_py(val, 5, 10))

In [10]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sig_inv_loss(max_error_tot):
    return (1 - sigmoid( (max_error_tot/50) - 10))

def sig_inv_loss_tf(max_error_tot):
    return (1 - tf.sigmoid( (max_error_tot/50) - 10)).eval()*0.5

def log_loss_tf(max_error_tot):
    return tf.log1p(max_error_tot*100).eval()


In [11]:
t = np.arange(0, 100000, 100.0)
for val in t:
    print (val, log_loss_tf(val))

0.0 0.0
100.0 9.21044036698
200.0 9.90353755129
300.0 10.3089859934
400.0 10.5966597328
500.0 10.8197982842
600.0 11.0021165077
700.0 11.1562648066
800.0 11.2897944136
900.0 11.4075760604
1000.0 11.5129354649
1100.0 11.6082447356
1200.0 11.6952553551
1300.0 11.7752974217
1400.0 11.8494048444
1500.0 11.9183972397
1600.0 11.9829353442
1700.0 12.0435595984
1800.0 12.1007176854
1900.0 12.1547846143
2000.0 12.2060776455
2100.0 12.2548675716
2200.0 12.3013873708
2300.0 12.3458389357
2400.0 12.388398369
2500.0 12.4292201968
2600.0 12.4684407561
2700.0 12.5061809417
2800.0 12.5425484536
2900.0 12.5776396502
3000.0 12.611541087
3100.0 12.6443308023
3200.0 12.6760793998
3300.0 12.7068509637
3400.0 12.7367038378
3500.0 12.7656912906
3600.0 12.7938620882
3700.0 12.8212609873
3800.0 12.8479291633
3900.0 12.8739045822
4000.0 12.8992223261
4100.0 12.9239148777
4200.0 12.9480123712
4300.0 12.9715428132
4400.0 12.9945322786
4500.0 13.017005084
4600.0 13.0389839424
4700.0 13.0604901013
4800.0 13.0815434